# 1. Recogida y limpieza de datos

Importamos las librerías necesarias. Además, permitimos mostrar más columnas y filas para trabajar con comodidad.

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)

Empezamos a extraer los datos de Understat con un ejemplo, el de La liga 2019/20.

In [2]:
url = 'https://understat.com/league/La_liga/2019'
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")
scripts = soup.find_all('script')

In [3]:
scripts

[<script>
 			var THEME = localStorage.getItem("theme") || 'DARK';
 			document.body.className = "theme-" + THEME.toLowerCase();
 		</script>,
 <script>
 	var datesData 	= JSON.parse('\x5B\x7B\x22id\x22\x3A\x2212026\x22,\x22isResult\x22\x3Atrue,\x22h\x22\x3A\x7B\x22id\x22\x3A\x22147\x22,\x22title\x22\x3A\x22Athletic\x20Club\x22,\x22short_title\x22\x3A\x22ATH\x22\x7D,\x22a\x22\x3A\x7B\x22id\x22\x3A\x22148\x22,\x22title\x22\x3A\x22Barcelona\x22,\x22short_title\x22\x3A\x22BAR\x22\x7D,\x22goals\x22\x3A\x7B\x22h\x22\x3A\x221\x22,\x22a\x22\x3A\x220\x22\x7D,\x22xG\x22\x3A\x7B\x22h\x22\x3A\x220.457129\x22,\x22a\x22\x3A\x221.03621\x22\x7D,\x22datetime\x22\x3A\x222019\x2D08\x2D16\x2020\x3A00\x3A00\x22,\x22forecast\x22\x3A\x7B\x22w\x22\x3A\x220.1496\x22,\x22d\x22\x3A\x220.3399\x22,\x22l\x22\x3A\x220.5105\x22\x7D\x7D,\x7B\x22id\x22\x3A\x2212023\x22,\x22isResult\x22\x3Atrue,\x22h\x22\x3A\x7B\x22id\x22\x3A\x22152\x22,\x22title\x22\x3A\x22Celta\x20Vigo\x22,\x22short_title\x22\x3A\x22CEL\x22\x7D,\x22a

Adecuamos los datos, buscando las partes que nos interesan y eliminando el resto.

In [4]:
buscar_json = ''

#Buscamos los datos del equipo
for el in scripts:
    if 'teamsData' in str(el):
        buscar_json = str(el).strip()

#Adecuamos los valores encontrados
inicio = buscar_json.index("('")+2
fin = buscar_json.index("')")
json_data = buscar_json[inicio:fin]

json_data = json_data.encode('utf8').decode('unicode_escape')

In [5]:
json_data

'{"138":{"id":"138","title":"Sevilla","history":[{"h_a":"a","xG":2.400440000000000129176669361186213791370391845703125,"xGA":0.1458790000000000086632923057550215162336826324462890625,"npxG":2.400440000000000129176669361186213791370391845703125,"npxGA":0.1458790000000000086632923057550215162336826324462890625,"ppda":{"att":256,"def":23},"ppda_allowed":{"att":327,"def":29},"deep":6,"deep_allowed":0,"scored":2,"missed":0,"xpts":2.90739999999999998436805981327779591083526611328125,"result":"w","date":"2019-08-18 18:00:00","wins":1,"draws":0,"loses":0,"pts":3,"npxGD":2.254561000000000259291255133575759828090667724609375},{"h_a":"a","xG":1.3921099999999999585753585051861591637134552001953125,"xGA":0.428331999999999990524912618639064021408557891845703125,"npxG":1.3921099999999999585753585051861591637134552001953125,"npxGA":0.428331999999999990524912618639064021408557891845703125,"ppda":{"att":181,"def":24},"ppda_allowed":{"att":238,"def":21},"deep":3,"deep_allowed":6,"scored":1,"missed":0,"xp

In [6]:
# Convertimos el json en un diccionario
data = json.loads(json_data)

In [7]:
data

{'138': {'id': '138',
  'title': 'Sevilla',
  'history': [{'h_a': 'a',
    'xG': 2.40044,
    'xGA': 0.145879,
    'npxG': 2.40044,
    'npxGA': 0.145879,
    'ppda': {'att': 256, 'def': 23},
    'ppda_allowed': {'att': 327, 'def': 29},
    'deep': 6,
    'deep_allowed': 0,
    'scored': 2,
    'missed': 0,
    'xpts': 2.9074,
    'result': 'w',
    'date': '2019-08-18 18:00:00',
    'wins': 1,
    'draws': 0,
    'loses': 0,
    'pts': 3,
    'npxGD': 2.2545610000000003},
   {'h_a': 'a',
    'xG': 1.39211,
    'xGA': 0.428332,
    'npxG': 1.39211,
    'npxGA': 0.428332,
    'ppda': {'att': 181, 'def': 24},
    'ppda_allowed': {'att': 238, 'def': 21},
    'deep': 3,
    'deep_allowed': 6,
    'scored': 1,
    'missed': 0,
    'xpts': 2.271,
    'result': 'w',
    'date': '2019-08-23 19:00:00',
    'wins': 1,
    'draws': 0,
    'loses': 0,
    'pts': 3,
    'npxGD': 0.963778},
   {'h_a': 'h',
    'xG': 1.50787,
    'xGA': 0.454026,
    'npxG': 1.50787,
    'npxGA': 0.454026,
    'ppda'

Obtenemos cada equipo con su ID correspondiente y el nombre de las columnas de lo que será nuestro DataFrame.

In [8]:
equipos = {}
for id in data.keys():
    equipos[id] = data[id]['title']

equipos

{'138': 'Sevilla',
 '140': 'Real Sociedad',
 '141': 'Espanyol',
 '142': 'Getafe',
 '143': 'Atletico Madrid',
 '146': 'Valencia',
 '147': 'Athletic Club',
 '148': 'Barcelona',
 '150': 'Real Madrid',
 '151': 'Levante',
 '152': 'Celta Vigo',
 '153': 'Real Betis',
 '154': 'Villarreal',
 '155': 'Granada',
 '156': 'Eibar',
 '157': 'Osasuna',
 '158': 'Alaves',
 '159': 'Leganes',
 '231': 'Real Valladolid',
 '239': 'Mallorca'}

In [9]:
columns = []
for id in data.keys():
    columns = list(data[id]['history'][0].keys())

columns

['h_a',
 'xG',
 'xGA',
 'npxG',
 'npxGA',
 'ppda',
 'ppda_allowed',
 'deep',
 'deep_allowed',
 'scored',
 'missed',
 'xpts',
 'result',
 'date',
 'wins',
 'draws',
 'loses',
 'pts',
 'npxGD']

In [10]:
# Obtenemos los datos de todos los equipos
tabla = {}
for id, equipo in equipos.items():
    datos_equipos = []
    for row in data[id]['history']:
        datos_equipos.append(list(row.values()))
    
    df = pd.DataFrame(datos_equipos, columns=columns)
    tabla[equipo] = df

In [11]:
#Mostramos un ejemplo de cómo quedaría hasta el momento
tabla["Valencia"].sample()

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
28,a,0.156776,2.02374,0.156776,2.02374,"{'att': 314, 'def': 16}","{'att': 277, 'def': 29}",1,12,0,3,0.1638,l,2020-06-18 20:00:00,0,0,1,0,-1.866964


In [12]:
#Adecuamos los valores del PPDA y OPPDA
for equipo, df in tabla.items():
    tabla[equipo]['PPDA'] = tabla[equipo]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    tabla[equipo]['OPPDA'] = tabla[equipo]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

#Mostramos cómo quedaría
tabla['Valencia'].sample()

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD,PPDA,OPPDA
16,h,2.28638,1.97126,2.28638,1.97126,"{'att': 365, 'def': 14}","{'att': 236, 'def': 22}",2,10,1,1,1.6356,d,2019-12-15 20:00:00,0,1,0,1,0.31512,26.071429,10.727273


Vamos a agrupar todos los valores de cada uno de los equipos de la liga. Según sean los datos, los sumamos o nos quedamos con el promedio.

In [13]:
sumar_columnas = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
promedio_columnas = ['PPDA', 'OPPDA']

In [14]:
frames = []
for equipo, df in tabla.items():
    datos_suma = pd.DataFrame(df[sumar_columnas].sum()).transpose()
    datos_media = pd.DataFrame(df[promedio_columnas].mean()).transpose()
    final_df = datos_suma.join(datos_media)
    final_df['team'] = equipo
    final_df['matches'] = len(df)
    frames.append(final_df)

datos_completos = pd.concat(frames)

datos_completos

,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,wins,draws,loses,pts,npxGD,PPDA,OPPDA,team,matches
0,55.155506,36.617301,49.952825,33.628775,238.0,148.0,54.0,34.0,65.7050,19.0,13.0,6.0,70.0,16.324050,8.411133,12.642357,Sevilla,38
0,50.915671,41.885105,47.199543,35.196378,231.0,192.0,56.0,48.0,57.1783,16.0,8.0,14.0,56.0,12.003165,8.649489,11.924641,Real Sociedad,38
0,37.002815,53.279324,34.029713,47.916092,148.0,191.0,27.0,58.0,41.7590,5.0,10.0,23.0,25.0,-13.886379,9.441301,10.059594,Espanyol,38
0,45.191234,43.023383,39.886634,38.563985,176.0,119.0,43.0,37.0,53.7878,14.0,12.0,12.0,54.0,1.322649,7.005995,6.302145,Getafe,38
0,58.700963,31.515192,52.739451,28.542603,264.0,162.0,51.0,27.0,72.3506,18.0,16.0,4.0,70.0,24.196848,9.686665,10.994450,Atletico Madrid,38
0,45.576255,59.073642,39.630147,52.384285,217.0,262.0,46.0,53.0,45.4291,14.0,11.0,13.0,53.0,-12.754138,14.086311,10.543273,Valencia,38
0,42.095595,52.419298,34.595139,47.216360,122.0,205.0,41.0,38.0,44.6548,13.0,12.0,13.0,51.0,-12.621221,9.713231,10.225018,Athletic Club,38
0,72.009449,37.823949,67.549781,32.620992,440.0,146.0,86.0,38.0,73.1381,25.0,7.0,6.0,82.0,34.928789,8.256988,17.961598,Barcelona,38
0,72.157218,33.146266,63.981449,31.659710,334.0,127.0,70.0,25.0,77.6868,26.0,9.0,3.0,87.0,32.321739,9.262202,14.476897,Real Madrid,38
0,49.427886,64.249355,45.711750,58.887593,177.0,273.0,47.0,53.0,42.8790,14.0,7.0,17.0,49.0,-13.175843,12.009367,8.825311,Levante,38


In [15]:
#Realizamos una serie de ajustes para mostrar mejor los datos
datos_completos = datos_completos[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'PPDA', 'OPPDA', 'deep', 'deep_allowed', 'xpts']]
datos_completos.sort_values('pts', ascending=False, inplace=True)
datos_completos.reset_index(inplace=True, drop=True)
datos_completos['posición'] = range(1,len(datos_completos)+1)
datos_completos.columns = ["Equipo", "PJ", "G", "E", "P", "GF", "GC", "Pts", "xG", "npxG", "xGA", "npxGA", "npxGDif", 'PPDA', 'OPPDA', "Pases finales", "Pases finales permitidos", "xPts", "Posición"]
datos_completos["Pases finales"] = datos_completos["Pases finales"]/datos_completos["PJ"]
datos_completos["Pases finales permitidos"] = datos_completos["Pases finales permitidos"]/datos_completos["PJ"]
orden_columnas = ["Posición", "Equipo", "PJ", "G", "E", "P", "GF", "GC", "Pts", "xG", "npxG", "xGA", "npxGA", "npxGDif", 'PPDA', 'OPPDA', "Pases finales", "Pases finales permitidos", "xPts"]
datos_completos = datos_completos[orden_columnas]
datos_completos

,Posición,Equipo,PJ,G,E,P,GF,GC,Pts,xG,npxG,xGA,npxGA,npxGDif,PPDA,OPPDA,Pases finales,Pases finales permitidos,xPts
0,1,Real Madrid,38,26.0,9.0,3.0,70.0,25.0,87.0,72.157218,63.981449,33.146266,31.659710,32.321739,9.262202,14.476897,8.789474,3.342105,77.6868
1,2,Barcelona,38,25.0,7.0,6.0,86.0,38.0,82.0,72.009449,67.549781,37.823949,32.620992,34.928789,8.256988,17.961598,11.578947,3.842105,73.1381
2,3,Sevilla,38,19.0,13.0,6.0,54.0,34.0,70.0,55.155506,49.952825,36.617301,33.628775,16.324050,8.411133,12.642357,6.263158,3.894737,65.7050
3,4,Atletico Madrid,38,18.0,16.0,4.0,51.0,27.0,70.0,58.700963,52.739451,31.515192,28.542603,24.196848,9.686665,10.994450,6.947368,4.263158,72.3506
4,5,Villarreal,38,18.0,6.0,14.0,63.0,49.0,60.0,65.606501,57.430565,48.589865,41.832685,15.597880,10.900010,12.067332,5.736842,6.000000,64.0527
5,6,Real Sociedad,38,16.0,8.0,14.0,56.0,48.0,56.0,50.915671,47.199543,41.885105,35.196378,12.003165,8.649489,11.924641,6.078947,5.052632,57.1783
6,7,Granada,38,16.0,8.0,14.0,52.0,45.0,56.0,45.021407,41.305021,43.439237,39.723106,1.581915,10.176947,7.772832,3.552632,5.184211,52.5969
7,8,Getafe,38,14.0,12.0,12.0,43.0,37.0,54.0,45.191234,39.886634,43.023383,38.563985,1.322649,7.005995,6.302145,4.631579,3.131579,53.7878
8,9,Valencia,38,14.0,11.0,13.0,46.0,53.0,53.0,45.576255,39.630147,59.073642,52.384285,-12.754138,14.086311,10.543273,5.710526,6.894737,45.4291
9,10,Osasuna,38,13.0,13.0,12.0,46.0,54.0,52.0,49.038488,43.092265,49.580451,45.864071,-2.771806,9.097908,8.506952,5.184211,5.026316,51.5097


In [16]:
#Eliminamos algunos decimales innecesarios
columnas_enteros = ['Posición', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'Pts']
datos_completos[columnas_enteros] = datos_completos[columnas_enteros].astype(int)
pd.options.display.float_format = '{:,.2f}'.format

In [17]:
datos_completos

,Posición,Equipo,PJ,G,E,P,GF,GC,Pts,xG,npxG,xGA,npxGA,npxGDif,PPDA,OPPDA,Pases finales,Pases finales permitidos,xPts
0,1,Real Madrid,38,26,9,3,70,25,87,72.16,63.98,33.15,31.66,32.32,9.26,14.48,8.79,3.34,77.69
1,2,Barcelona,38,25,7,6,86,38,82,72.01,67.55,37.82,32.62,34.93,8.26,17.96,11.58,3.84,73.14
2,3,Sevilla,38,19,13,6,54,34,70,55.16,49.95,36.62,33.63,16.32,8.41,12.64,6.26,3.89,65.70
3,4,Atletico Madrid,38,18,16,4,51,27,70,58.70,52.74,31.52,28.54,24.20,9.69,10.99,6.95,4.26,72.35
4,5,Villarreal,38,18,6,14,63,49,60,65.61,57.43,48.59,41.83,15.60,10.90,12.07,5.74,6.00,64.05
5,6,Real Sociedad,38,16,8,14,56,48,56,50.92,47.20,41.89,35.20,12.00,8.65,11.92,6.08,5.05,57.18
6,7,Granada,38,16,8,14,52,45,56,45.02,41.31,43.44,39.72,1.58,10.18,7.77,3.55,5.18,52.60
7,8,Getafe,38,14,12,12,43,37,54,45.19,39.89,43.02,38.56,1.32,7.01,6.30,4.63,3.13,53.79
8,9,Valencia,38,14,11,13,46,53,53,45.58,39.63,59.07,52.38,-12.75,14.09,10.54,5.71,6.89,45.43
9,10,Osasuna,38,13,13,12,46,54,52,49.04,43.09,49.58,45.86,-2.77,9.10,8.51,5.18,5.03,51.51
